<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

Install libraries

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} numpy pandas  matplotlib 

!conda install -c conda-forge geopy beautifulsoup4 scikit-learn geocoder folium=0.5.0 --yes
print('Packages installed.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - matplotlib
    - numpy
    - pandas


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\chitt\Anaconda3

  added / updated specs:
    - beautifulsoup4
    - folium=0.5.0
    - geocoder
    - geopy
    - scikit-learn


The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Packages installed.


Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup
import geocoder

print('Libraries imported.')

Libraries imported.


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

print(soup.prettify()) 

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"980990e8-caa6-4910-84a1-d2da1d8aa4ca","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958430791,"wgRevisionId":958430791,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [5]:

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood','Latitude','Longtitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longtitude


In [6]:
lat_lng_coords = None

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g=geocoder.arcgis('{}, Toronto,Ontario'.format(code))
    lat_lng_coords =g.latlng

    neighbors = neighbors.append({ 'PostalCode': code[:-1],
                                   'Borough': borough[:-1],
                                   'Neighborhood': neighborhood_name[:-1],
                                   'Latitude':lat_lng_coords[0],
                                   'Longtitude':lat_lng_coords[1]}, ignore_index=True)

In [7]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


In [8]:
neighbors.shape

(103, 5)

In [18]:
neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
lat = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
lng = neighbors.loc[0, 'Longtitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               lat, 
                                                               lng))


CLIENT_ID='X42LYOEZGJAW2WD35SA4TB2LESK4BOMBKXPBM4QLGQY40NDZ'
CLIENT_SECRET='FTRC4MGDBI4LDG5VBA0NMAU1DXS14SPIC1UTP3ZILTAJEMTP'
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods are 43.75293455500008, -79.33564142299997.


{'meta': {'code': 200, 'requestId': '5ed1424ec8cff2001bff9b24'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Sunnybrook - York Mills',
  'headerFullLocation': 'Sunnybrook - York Mills, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.75743455950008,
    'lng': -79.32942319651914},
   'sw': {'lat': 43.74843455050008, 'lng': -79.3418596494808}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeled

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longtitude'])

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue, Humber Valley Village

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto, Broadview North (Old East York)

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinit

In [25]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(206, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods\n,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods\n,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village\n,43.728102,-79.311890,Brookbanks Park,43.751976,-79.332140,Park
3,Victoria Village\n,43.728102,-79.311890,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,"Regent Park, Harbourfront\n",43.650964,-79.353041,Brookbanks Park,43.751976,-79.332140,Park


In [26]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,2,2,2,2,2,2
"Alderwood, Long Branch\n",2,2,2,2,2,2
"Bathurst Manor, Wilson Heights, Downsview North\n",2,2,2,2,2,2
Bayview Village\n,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East\n",2,2,2,2,2,2
...,...,...,...,...,...,...
"Willowdale, Willowdale West\n",2,2,2,2,2,2
Woburn\n,2,2,2,2,2,2
Woodbine Heights\n,2,2,2,2,2,2


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 2 uniques categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Food & Drink Shop,Park
0,Parkwoods\n,0,1
1,Parkwoods\n,1,0
2,Victoria Village\n,0,1
3,Victoria Village\n,1,0
4,"Regent Park, Harbourfront\n",0,1


In [29]:
# examine the new dataframe size.

toronto_onehot.shape

(206, 3)

In [30]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Food & Drink Shop,Park
0,Agincourt\n,0.5,0.5
1,"Alderwood, Long Branch\n",0.5,0.5
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.5,0.5
3,Bayview Village\n,0.5,0.5
4,"Bedford Park, Lawrence Manor East\n",0.5,0.5
...,...,...,...
94,"Willowdale, Willowdale West\n",0.5,0.5
95,Woburn\n,0.5,0.5
96,Woodbine Heights\n,0.5,0.5
97,York Mills West\n,0.5,0.5


In [31]:
# confirm the new size

toronto_grouped.shape

(99, 3)

In [32]:

# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Alderwood, Long Branch
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bayview Village
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Bedford Park, Lawrence Manor East
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Berczy Park
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Birch Cliff, Cliffside West
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Brockton, Parkdale Village, Exhibition Place
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Business reply mail Processing Centre, South

1               Park   0.5


----The Beaches
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----The Danforth West, Riverdale
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----The Kingsway, Montgomery Road, Old Mill North
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Thorncliffe Park
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Toronto Dominion Centre, Design Exchange
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----University of Toronto, Harbord
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Upper Rouge
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----Victoria Village
----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5


----West Deane Park, Princess 

In [34]:

# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
0,Agincourt\n,Park,Food & Drink Shop
1,"Alderwood, Long Branch\n",Park,Food & Drink Shop
2,"Bathurst Manor, Wilson Heights, Downsview North\n",Park,Food & Drink Shop
3,Bayview Village\n,Park,Food & Drink Shop
4,"Bedford Park, Lawrence Manor East\n",Park,Food & Drink Shop


In [45]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

C:\Users\chitt\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.728102,-79.311890,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,NaN,NaN,NaN


In [53]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longtitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[pi-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

In [43]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
